## 2.1 使用多个界定符分割字符串

### string对象的split（）方法只适应于非常简单的字符串分割情形，它并不允许有多个分隔符，或者分隔符周围有空格的情况。当你需要更加灵活的切割字符串的时候，最好使用re.split()方法。

In [1]:
line = 'asdf fjdk; afed, fjek,asdf, foo'

In [2]:
import re

In [3]:
re.split(r'[;,\s]\s*',line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

### \s表示任意的空白字符（等价于 [\t\n\r\f])

### 当是使用re.split()的时候，需要注意，pattern里是否包含括号捕获分组。如果使用了捕获分组，那么被匹配的文本也将出现在结果列表中。

In [4]:
fields = re.split(r'(;|c|\s)\s*',line)

In [5]:
print(fields) ## 会把括号捕获分组里的东西也抓取到

['asdf', ' ', 'fjdk', ';', 'afed,', ' ', 'fjek,asdf,', ' ', 'foo']


In [6]:
values = fields[::2]
delimiters = fields[1::2]

In [7]:
values

['asdf', 'fjdk', 'afed,', 'fjek,asdf,', 'foo']

In [8]:
delimiters

[' ', ';', ' ', ' ']

In [9]:
''.join(v+d for v,d in zip(values, delimiters))

'asdf fjdk;afed, fjek,asdf, '

### 如果你不想保留分割字符串到结果列表里面去，但仍然需要使用到括号来分组正则表达式的话，确保你的分组是非捕获分组，形如(?:...)，例如：

In [11]:
re.split(r'(?:,|;|\s)\s*',line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

## 2.2 字符串开头或结尾匹配

### 检查字符串开头或者结尾的简单反发是通过str.startswith() 或者str.endswith()

In [13]:
filename = 'spam.txt'

In [14]:
filename.endswith('txt')

True

In [15]:
filename.startswith("fffff")

False

In [16]:
url = 'http://www.python.org'

In [17]:
url.startswith("http")

True

In [18]:
### 还可以检查多种可能，只需要将每种情况组合成一个元组，然后传入就ok


In [19]:
import os

In [20]:
filename = os.listdir('.')

In [21]:
filename

['.git',
 '.ipynb_checkpoints',
 'Chapter_1.ipynb',
 'Chapter_2.ipynb',
 'README.md']

In [24]:
any(name.endswith(('py','md'))  for name in filename)

True

In [25]:
from urllib.request import urlopen

In [26]:
def read_data(name):
    if name.startswith(('http:','https:','ftp:')):
        return urlopen(name).read()
    else:
        with open(name) as f:
            return f.read()

In [28]:
#read_data(r"http:\\www.baidu.com")

In [30]:
if any(file.endswith(('.h','.py','c','md')) for file in filename):
    print("ok find it")
else:
    print("nothing found")

ok find it


## 2.3 用Shell通配符匹配字符串

In [31]:
### fnmatch 使用unix shell的通配符，来对文本进行匹配，是一种比正则简单，但比简单的字符串方法要牛一点。

In [32]:
from fnmatch import fnmatch, fnmatchcase

In [33]:
fnmatch('foo.txt','*.txt')

True

In [34]:
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']

In [35]:
[name for name in names if fnmatch(name, 'Dat*.csv')]

['Dat1.csv', 'Dat2.csv']

## 2.4 字符串匹配和搜索

In [36]:
text1 = '11/27/2012'

In [37]:
import re

In [53]:
ret = re.match(r'\d+/\d+/\d+',text1)

In [54]:
ret.groups()

()

### 使用（）可以对pattern进行分组，分组里的内容会被匹配抓到re.groups()里。

### findall会搜索文本，然后以列表的形式返回所有的匹配。 如果想以迭代方法匹配，可以用finditer()。例如：


In [56]:
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'

In [59]:
for m in re.finditer(r'\d+/\d+/\d+', text):
    print(m.group())
    
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')
for m in datepat.finditer( text):
    print(m.groups())

11/27/2012
3/13/2013
('11', '27', '2012')
('3', '13', '2013')


### However，杀鸡焉用牛刀？ 普通的“固定”字符匹配，完全可以用str.find, str.startswith, str.endswith来匹配。

## 2.6 字符串忽略大小写的搜索替换

In [60]:
import re

In [61]:
text = 'UPPER PYTHON, lower python, Mixed Python'

In [71]:
# m = re.match('UPPER', text)
# m.group(0)

re.findall('python',text,flags=re.IGNORECASE)

['PYTHON', 'python', 'Python']

In [72]:
re.sub('python','snake',text,flags=re.IGNORECASE)

'UPPER snake, lower snake, Mixed snake'

In [73]:
def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isupper():
            return word.upper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word
    return replace

In [76]:
re.sub('python', matchcase('snake'),text, flags=re.IGNORECASE)

'UPPER SNAKE, lower snake, Mixed Snake'

## 2.7 最短匹配模式

### 在匹配的pattern，例如(.*？),*后面加个？，表示懒惰模式。

In [77]:
text2 = 'Computer says "no." Phone says "yes."'

In [79]:
re.findall(r'"(.*?)"',text2)

['no.', 'yes.']

## 2.8 多行匹配模式

In [80]:
### 这个问题很典型的出现在，你想用.来匹配任意字符，但是又忘记.无法匹配换行符... 

In [93]:
comment = re.compile(r'/\*(.*?)\*/')

In [94]:
text1 = '/* this is a comment */'
text2 = '''/* this is a
multiline comment */
'''


In [95]:
comment.findall(text1)

[' this is a comment ']

In [96]:
re.findall(r'/\*((?:.|\n)*?)\*/',text2)

[' this is a\nmultiline comment ']

In [97]:
comment_new = re.compile(r'/\*(.*?)\*/',flags=re.DOTALL)
comment_new.findall(text2)

[' this is a\nmultiline comment ']

## 2.9 将Unicode文本标准化

In [98]:
### pass...

## 2.10  在正则式中使用Unicode

In [99]:
import re

In [100]:
num = re.compile(r'\d+')

In [101]:
num.match('123')

<_sre.SRE_Match object; span=(0, 3), match='123'>

In [102]:
num.match('\u0661\u0662\u0663')

<_sre.SRE_Match object; span=(0, 3), match='١٢٣'>

In [103]:
aa = '\u0661\u0662\u0663'

b'\\u0661\\u0662\\u0663'


## 2.11 删除字符串中不需要的字符

### strip方法可以删除开始或者结尾的字符。 lstrip() rstrip()，分别从左和从优执行删除操作。，默认是取出空白字符，也可以指定其他字符。


In [112]:
s = ' hello world \n'

In [115]:
# print(s)
s.strip()

'hello world'

In [116]:
s.lstrip()

'hello world \n'

In [117]:
s.rstrip()

' hello world'

In [118]:
t = '-----hello====='

In [123]:
t.strip('=-,?') 

'hello'

## strip的参数，是一个字符串，字符串里可以包含多个需要去除的字符。

In [ ]:
# with open(filename) as f:
#     lines = (line.strip() for line in f)
#     for line in lines:
#         print(line)



## 2.13 字符串对齐

In [125]:
'{^20.2f} {>10}'.format(1.2, 3)

KeyError: '^20'